ПАКЕТ SURPRISE

используйте данные MovieLens 1M

можно использовать любые модели из пакета

получите RMSE на тестовом сете 0.87 и ниже

Комментарий преподавателя :

В ДЗ на датасет 1М может не хватить RAM. Можно сделать на 100K. Качество RMSE предлагаю считать на основе CrossValidation (5 фолдов), а не отложенном датасете.

Файл Практика.ipynb лежит в слаке в чате aml7ds9

In [236]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer, TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

In [150]:
users = pd.read_table('users.dat', sep='::', header=None, names=['user_id', 'gender', 'age', 'occupation', 'zip'])
users.head()

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


,user_id,gender,age,occupation,zip
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [151]:
users.shape

(6040, 5)

In [152]:
movies = pd.read_table('movies.dat', sep='::', header=None, names=['movie_id', 'title', 'genres'])
movies.head()

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [153]:
movies.shape

(3883, 3)

In [154]:
ratings = pd.read_table('ratings.dat', sep='::', header=None, names=['user_id', 'movie_id', 'rating', 'timestamp'])
ratings.head()

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [155]:
ratings.shape

(1000209, 4)

In [218]:
df_user1 = ratings.merge(movies, on='movie_id', how='left')[ratings['user_id'] == 1]

In [219]:
df_user1.head()

,user_id,movie_id,rating,timestamp,title,genres
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama
1,1,661,3,978302109,James and the Giant Peach (1996),Animation|Children's|Musical
2,1,914,3,978301968,My Fair Lady (1964),Musical|Romance
3,1,3408,4,978300275,Erin Brockovich (2000),Drama
4,1,2355,5,978824291,"Bug's Life, A (1998)",Animation|Children's|Comedy


In [220]:
df_user1.shape

(53, 6)

In [221]:
y = df_user1['rating']

In [222]:
def change_string(s):
    return ' '.join(s.replace(' ', '').replace('-', '').split('|'))

In [223]:
movie_genres = [change_string(g) for g in df_user1.genres.values]

In [224]:
tfidf_vectorizer = TfidfVectorizer()
X_train_tfidf = tfidf_vectorizer.fit_transform(movie_genres)

In [225]:
df1 = pd.DataFrame(X_train_tfidf.toarray(), columns = tfidf_vectorizer.get_feature_names())
res = pd.concat([df_user1, df1], axis=1)

In [226]:
res.head()

,user_id,movie_id,rating,timestamp,title,genres,action,adventure,animation,children,comedy,crime,drama,fantasy,musical,romance,scifi,thriller,war
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama,0.0,0.0,0.000000,0.000000,0.00000,0.0,1.0,0.0,0.000000,0.000000,0.0,0.0,0.0
1,1,661,3,978302109,James and the Giant Peach (1996),Animation|Children's|Musical,0.0,0.0,0.563517,0.535932,0.00000,0.0,0.0,0.0,0.628670,0.000000,0.0,0.0,0.0
2,1,914,3,978301968,My Fair Lady (1964),Musical|Romance,0.0,0.0,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.599769,0.800173,0.0,0.0,0.0
3,1,3408,4,978300275,Erin Brockovich (2000),Drama,0.0,0.0,0.000000,0.000000,0.00000,0.0,1.0,0.0,0.000000,0.000000,0.0,0.0,0.0
4,1,2355,5,978824291,"Bug's Life, A (1998)",Animation|Children's|Comedy,0.0,0.0,0.563517,0.535932,0.62867,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0


In [227]:
X = res.drop(['rating', 'title', 'genres', 'user_id', 'movie_id', 'timestamp'], axis=1)

In [228]:
X.head()

,action,adventure,animation,children,comedy,crime,drama,fantasy,musical,romance,scifi,thriller,war
0,0.0,0.0,0.000000,0.000000,0.00000,0.0,1.0,0.0,0.000000,0.000000,0.0,0.0,0.0
1,0.0,0.0,0.563517,0.535932,0.00000,0.0,0.0,0.0,0.628670,0.000000,0.0,0.0,0.0
2,0.0,0.0,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.599769,0.800173,0.0,0.0,0.0
3,0.0,0.0,0.000000,0.000000,0.00000,0.0,1.0,0.0,0.000000,0.000000,0.0,0.0,0.0
4,0.0,0.0,0.563517,0.535932,0.62867,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0


In [293]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()

In [294]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5)

In [295]:
model.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [297]:
y_pred = model.predict(X_test)

In [298]:
from sklearn import metrics
print(np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

0.7566118862156564
